In [31]:
import numpy as np
import pandas as pd
from collections import Counter
from skimage import io, filters, morphology, segmentation, img_as_ubyte, transform, color
import matplotlib.pyplot as plt
from skimage.draw import polygon

from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import joblib

import os
import shutil
import pandas as pd

In [32]:
np.random.seed(1)

#Data Selection and Balancing for Diagnostic Analysis: Creating a Balanced Dataset with Pandas

1. The make_df function reads the metadata from a dataset stored in a CSV file called "metadata.csv" and creates a pandas DataFrame (df) with the columns "patient_id", "img_id", and "diagnostic". The function then creates a new DataFrame (new_df) by selecting only the columns "patient_id", "img_id", and "diagnostic" from df.
2. In the new_df DataFrame, a new column called "healthy" is created using the np.where function. The "healthy" column is assigned a value of 1 if the corresponding "diagnostic" column value is "NEV" (indicating a healthy diagnosis), and 0 otherwise.
3. The select_data function takes the new_df DataFrame as input. It creates a new DataFrame called final_data by filtering rows where the "healthy" column value is 1, indicating a healthy diagnosis.
4. Another DataFrame called filtered_data is created by filtering rows where the "healthy" column value is 0, indicating a non-healthy diagnosis.
5. The sample function is used on the filtered_data DataFrame to randomly select 244 rows (representing non-healthy diagnoses) using a random state of 42. These randomly selected rows are stored in the random_rows DataFrame.
6. The pd.concat function is used to concatenate (pd.concat([final_data, random_rows])) the final_data DataFrame and the random_rows DataFrame, resulting in a new DataFrame that combines the healthy and randomly selected non-healthy rows.
7. The index of the combined DataFrame is set to "patient_id" using the set_index function.
8. The sample function is used again on the combined DataFrame with frac=1 to shuffle the rows randomly.
9. The resulting shuffled DataFrame is returned as final_data from the select_data function.
10 Finally, the final_data DataFrame is printed.

The overall purpose of the code is to create a pandas DataFrame (final_data) that represents a selected subset of the original dataset. It ensures that the "healthy" diagnosis ("NEV") is included in the final DataFrame and randomly selects a certain number of non-healthy diagnoses from the remaining data. The resulting DataFrame is shuffled to provide a random order of the data. This process aims to create a balanced dataset for further analysis or modeling purposes, where both healthy and non-healthy data points are included.

In [33]:
def make_df():
  path = os.path.join(os.getcwd(), "metadata.csv")
  df = pd.read_csv(path)
  new_df = df[["patient_id", "img_id", "diagnostic"]]

  new_df["healthy"] = np.where(new_df["diagnostic"] == "NEV", 1, 0) 
  return new_df 


def select_data(new_df):
  final_data = new_df[new_df["healthy"] == 1]
  filtered_data = new_df[new_df["healthy"] == 0]

  random_rows = filtered_data.sample(n = 244, random_state=42)
  final_data = pd.concat([final_data, random_rows])
  final_data = final_data.set_index("patient_id")

  final_data = final_data.sample(frac=1)

  return final_data


final_data = select_data(make_df())

final_data

C:\Users\dubst\AppData\Local\Temp\ipykernel_23564\1341983333.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["healthy"] = np.where(new_df["diagnostic"] == "NEV", 1, 0)


,img_id,diagnostic,healthy
patient_id,,,
PAT_520,PAT_520_983_221.png,BCC,0
PAT_1934,PAT_1934_3890_306.png,NEV,1
PAT_244,PAT_244_374_726.png,NEV,1
PAT_119,PAT_119_181_684.png,BCC,0
PAT_972,PAT_972_1843_756.png,MEL,0
...,...,...,...
PAT_1392,PAT_1392_1352_828.png,SEK,0
PAT_621,PAT_621_1182_456.png,NEV,1
PAT_872,PAT_872_1707_638.png,BCC,0


In [34]:
def slic_samples(img):
  new_img = img.copy()
  new_img = new_img[:, :, :3]

  foreground_mask = np.all(new_img != [0, 0, 0], axis=-1)

  segments = segmentation.slic(new_img * foreground_mask[..., np.newaxis], n_segments=36, compactness=3)

  mean_colours = np.zeros((np.max(segments)+1, 3))

  for label in enumerate(np.unique(segments)):
    mask = segments == label[1]
    mean_colours[label[0], :] = new_img[mask].mean(axis=0)

  palette_height, palette_width = 50, 300
  colours = mean_colours[np.all(mean_colours, axis=1)]
  color_palette = np.zeros((palette_height, len(colours), 3))

  for i in range(len(colours)):
    color_palette[:, i, :] = colours[i]

  return color_palette

In [35]:
def make_datasample(img, name):
  # asym = check_asymmetry(img)
  col = slic_samples(img)
  common_shape = (50, 27, 3)

  col = np.pad(col, [(0, common_shape[0] - col.shape[0]),
                                  (0, common_shape[1] - col.shape[1]),
                                  (0, common_shape[2] - col.shape[2])], mode='constant')

  col = col.ravel()
  x = col

  if (final_data[final_data["img_id"] == name]["healthy"] == 1).bool():
    y = 1
  else:
    y = 0
  
  return [x, y]

In [36]:
def build_datasample_new():
  path = os.path.join(os.getcwd(), "segmented_photos")
  arr = []

  for i in os.listdir(path):
    image = io.imread(os.path.join(path, i))
    image = transform.resize(image, (200, 200), anti_aliasing=True)

    arr.append(make_datasample(image, i))

  np.random.shuffle(arr)
  return arr


arr_col = build_datasample_new()
print(len(arr_col))


279


Then we want to calculate asymmetry coefficients by making histograms of a lesion and computing the Bhattacharyya coefficient.

In [37]:
def make_projections(bin_img):
  binary_image = bin_img.copy()

  vertical = np.sum(binary_image, 0)
  horizontal = np.sum(binary_image, 1)

  return vertical, horizontal

In [38]:
def calc_correlation(vertical, horizontal):
  # Normalize the histograms to ensure they represent probability distributions
    hist1_normalized = vertical / np.sum(vertical)
    hist2_normalized = horizontal / np.sum(horizontal)

    # Compute the Bhattacharyya coefficient
    bc = np.sum(np.sqrt(hist1_normalized * hist2_normalized))

    # Compute the Bhattacharyya distance
    bd = -np.log(bc)

    return bd

In [39]:
def check_asymmetry(masked_img):
  vert, horiz = make_projections(masked_img.astype("double"))
  corr = calc_correlation(vert, horiz)

  return corr

In [40]:
def make_datasample_symetry(img, name):
  asym = check_asymmetry(img)
  x = [asym]

  if (final_data[final_data["img_id"] == name]["healthy"] == 1).bool():
    y = 1
  else:
    y = 0
  
  return [x, y]

In [41]:
def build_datasample_asym():
  healthy_dir = os.path.join(os.getcwd(), "healthy")
  unheatlhy_dir = os.path.join(os.getcwd(), "unhealthy")

  arr = []

  for i in os.listdir(healthy_dir):
    image = io.imread(os.path.join(healthy_dir, i))
    image = transform.resize(image, (200, 200), anti_aliasing=True)

    arr.append(make_datasample_symetry(image, i))
  
  for i in os.listdir(unheatlhy_dir):
    image = io.imread(os.path.join(unheatlhy_dir, i))
    image = transform.resize(image, (200, 200), anti_aliasing=True)

    arr.append(make_datasample_symetry(image, i))

  np.random.shuffle(arr)
  return arr

arr_cor = build_datasample_asym()
print(len(arr_cor))

279


Here we define a function **separate_data** for splitting the data into training input samples and target values <br>
and an **evaluate** function for getting the metrics' scores.

In [42]:
def separate_data(arr):
  x = list()
  y = list()

  for i in arr:
    x.append(i[0])
    y.append(i[1])

  return (x, y)

In [43]:
def evaluate(model, test_features, test_labels):
  predictions = model.predict(test_features)
  errors = abs(predictions - test_labels)
  
  accuracy = accuracy_score(test_labels, predictions)
  precision = precision_score(test_labels, predictions)
  f1_score_res = f1_score(test_labels, predictions)
  recall = recall_score(test_labels, predictions)
  auc = roc_auc_score(test_labels, predictions)

  print(f'Model Performance {type(model).__name__}')
  print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
  print('Accuracy = {:0.4f}%.'.format(accuracy*100))
  print('Precision = {:0.4f}%.'.format(precision*100))
  print('F1 Score = {:0.4f}%.'.format(f1_score_res*100))
  print('Recall = {:0.4f}%.'.format(recall*100))
  print('AUC = {:0.4f}%.'.format(auc*100))
  
    
  return (np.mean(errors), accuracy, precision, f1_score_res, recall, auc)

Then we run final_training function, in order to see, how the different models give output, regarding to such metrics, as: <br> 
`Average Error`<br> 
`Accuracy`<br> 
`Precision`<br> 
`F1 Score`<br> 
`Recall`<br> 
`AUC`<br> 
`Cross-validation`

In [44]:
def final_training(arr, n_folds, title):

  x, y = separate_data(arr)

  x = np.array(x)
  y = np.array(y)
  
  (train_col, test_col, train_lab, test_lab) = train_test_split(
	x, y, test_size=0.25, random_state=42) 

  kf = KFold(n_splits=n_folds)

  clf = NearestCentroid()
  neigh = KNeighborsClassifier(n_neighbors=3)
  rndF = RandomForestClassifier()
  classifier = LogisticRegression(max_iter=207,random_state = 0, penalty = None)

  result_clf = cross_val_score(clf, train_col, train_lab, cv=n_folds)
  result_neigh = cross_val_score(neigh, train_col, train_lab, cv=n_folds)
  result_rndF = cross_val_score(rndF, train_col, train_lab, cv=n_folds)
  result_classifier = cross_val_score(classifier, train_col, train_lab, cv=n_folds)

  clf.fit(train_col, train_lab)
  neigh.fit(train_col, train_lab)
  rndF.fit(train_col, train_lab)
  classifier.fit(train_col, train_lab)
  
  eval_clf = evaluate(clf, test_col, test_lab); print(f"Cross-val: {np.mean(result_clf)}"); print("")
  eval_neigh = evaluate(neigh, test_col, test_lab); print(f"Cross-val: {np.mean(result_neigh)}"); print("")
  eval_rndF = evaluate(rndF, test_col, test_lab); print(f"Cross-val: {np.mean(result_rndF)}"); print("")
  eval_class = evaluate(classifier, test_col, test_lab); print(f"Cross-val: {np.mean(result_classifier)}"); print("")

  # Create a list of labels for the models
  models = ['NC', 'KNN', 'RFC', 'LR']

  # Create a list of metrics for each model
  average_errors = [eval_clf[0], eval_neigh[0], eval_rndF[0], eval_class[0]]
  accuracies = [eval_clf[1], eval_neigh[1], eval_rndF[1], eval_class[1]]
  precisions = [eval_clf[2], eval_neigh[2], eval_rndF[2], eval_class[2]]
  f1_scores = [eval_clf[3], eval_neigh[3], eval_rndF[3], eval_class[3]]
  recalls = [eval_clf[4], eval_neigh[4], eval_rndF[4], eval_class[4]]
  aucs = [eval_clf[5], eval_neigh[5], eval_rndF[5], eval_class[5]]
  cross_vals = [np.mean(result_clf), np.mean(result_neigh), np.mean(result_rndF), np.mean(result_classifier)]

  # Plotting the histograms
  fig, axs = plt.subplots(2, 4, figsize=(12, 5))
  axs = axs.flatten()

  # Histogram for Average Error
  axs[0].bar(models, average_errors)
  axs[0].set_title('Average Error')
  axs[0].set_ylabel('Degrees')

  # Histogram for Accuracy
  axs[1].bar(models, accuracies)
  axs[1].set_title('Accuracy')
  axs[1].set_ylabel('Percentage')

  # Histogram for Precision
  axs[2].bar(models, precisions)
  axs[2].set_title('Precision')
  axs[2].set_ylabel('Percentage')

  # Histogram for F1 Score
  axs[3].bar(models, f1_scores)
  axs[3].set_title('F1 Score')
  axs[3].set_ylabel('Percentage')

  # Histogram for Recall
  axs[4].bar(models, recalls)
  axs[4].set_title('Recall')
  axs[4].set_ylabel('Percentage')

  # Histogram for AUC
  axs[5].bar(models, aucs)
  axs[5].set_title('AUC')
  axs[5].set_ylabel('Percentage')

  axs[6].bar(models, cross_vals)
  axs[6].set_title('Cross-Validation')
  axs[6].set_ylabel('Value')

  fig.suptitle(title)

  return fig

After that, we define a function called **best_random_forest** to create a model, which is based on the parameters obtained from Grid and Random Searches.

In [29]:
def best_random_forest(arr, n_folds, title):
  x, y = separate_data(arr)

  x = np.array(x)
  y = np.array(y)
  
  (train_col, test_col, train_lab, test_lab) = train_test_split(
	x, y, test_size=0.25, random_state=42) 

  kf = KFold(n_splits=n_folds)
  
  rndF = RandomForestClassifier(n_estimators = 1200, min_samples_split=5, min_samples_leaf=3, max_features='sqrt', max_depth=30, bootstrap=True)
  result_rndF = cross_val_score(rndF, train_col, train_lab, cv=n_folds)
  
  rndF.fit(train_col, train_lab)

  # joblib.dump(rndF, 'C:\\Users\\dubst\\Desktop\\DataScience\\Project 2\\fyp2023\\random_forest_model_cor.pkl')

  score_rndF = rndF.score(test_col, test_lab)

  eval_rndF = evaluate(rndF, test_col, test_lab)

  fig, axs = plt.subplots(2, 4, figsize=(12, 5))
  axs = axs.flatten()

  # Histogram for Average Error
  axs[0].bar("Average Error", eval_rndF[0])
  axs[0].set_title(eval_rndF[0])

  axs[1].bar("Accuracy", eval_rndF[1])
  axs[1].set_title(eval_rndF[1])

  axs[2].bar("Precision", eval_rndF[2])
  axs[2].set_title(eval_rndF[2])

  axs[3].bar("F1 Score", eval_rndF[3])
  axs[3].set_title(eval_rndF[3])

  axs[4].bar("Recall", eval_rndF[4])
  axs[4].set_xlabel(eval_rndF[4])

  axs[5].bar("AUC", eval_rndF[5])
  axs[5].set_xlabel(eval_rndF[5])

  axs[6].bar("Cross-Validation", np.mean(result_rndF))
  axs[6].set_xlabel(np.mean(result_rndF))
  
  fig.suptitle(title)

  return fig

And the same is for the regular random forest with default parameters

In [54]:
def regular_random_forest(arr, n_folds):
  x, y = separate_data(arr)

  x = np.array(x)
  y = np.array(y)
  
  (train_col, test_col, train_lab, test_lab) = train_test_split(
	x, y, test_size=0.25, random_state=42) 

  kf = KFold(n_splits=n_folds)
  
  rndF = RandomForestClassifier()
  result_rndF = cross_val_score(rndF, train_col, train_lab, cv=n_folds)
  
  rndF.fit(train_col, train_lab)

  # joblib.dump(rndF, 'C:\\Users\\dubst\\Desktop\\DataScience\\Project 2\\fyp2023\\random_forest_model_cor.pkl')

  score_rndF = rndF.score(test_col, test_lab)

  eval_rndF = evaluate(rndF, test_col, test_lab)

  fig, axs = plt.subplots(2, 4, figsize=(12, 5))
  axs = axs.flatten()

  # Histogram for Average Error
  axs[0].bar("Average Error", eval_rndF[0])
  axs[0].set_title(eval_rndF[0])

  axs[1].bar("Accuracy", eval_rndF[1])
  axs[1].set_title(eval_rndF[1])

  axs[2].bar("Precision", eval_rndF[2])
  axs[2].set_title(eval_rndF[2])

  axs[3].bar("F1 Score", eval_rndF[3])
  axs[3].set_title(eval_rndF[3])

  axs[4].bar("Recall", eval_rndF[4])
  axs[4].set_xlabel(eval_rndF[4])

  axs[5].bar("AUC", eval_rndF[5])
  axs[5].set_xlabel(eval_rndF[5])

  axs[6].bar("Cross-Validation", np.mean(result_rndF))
  axs[6].set_xlabel(np.mean(result_rndF))
  

  return fig, eval_rndF, np.mean(result_rndF)

And we would like to store all the features in a .csv file

In [49]:
def make_csv_features(path):

  healthy_dir = os.path.join(os.getcwd(), "healthy")
  unhealthy_dir = os.path.join(os.getcwd(), "unhealthy")
  df = pd.DataFrame([])

  for i in os.listdir(healthy_dir):
    image = io.imread(os.path.join(healthy_dir, i))[:, :, :3]
    image = transform.resize(image, (200, 200), anti_aliasing=True)

    colour = make_datasample(image, i)
    asym = make_datasample_symetry(image, i)
    df["img_id"] = i 
    df["colour"] = colour[0]

    print(colour[0])
    df["asymmetry coef"] = asym[0][0]
    df["healthy"] = asym[1]

  for i in os.listdir(unhealthy_dir):
    image = io.imread(os.path.join(unhealthy_dir, i))[:, :, :3]
    image = transform.resize(image, (200, 200), anti_aliasing=True)

    colour = make_datasample(image, i)
    asym = make_datasample_symetry(image, i)
    df["img_id"] = i 
    df["colour"] = colour[0]
    df["asymmetry coef"] = asym[0][0]
    df["healthy"] = asym[1]

  df.to_csv(os.path.join(path, "output.csv"))

And here is the function for comparing the models

In [53]:
def compare_models(evals1, evals2, cros1, cros2):
  fig, axs = plt.subplots(2, 4, figsize=(12, 5))
  axs = axs.flatten()

  # Histogram for Average Error
  axs[0].bar("New", evals1[0])
  axs[0].set_title("Average Error")
  axs[0].bar("Old", evals2[0])
  axs[0].set_title("Average Error")

  axs[1].bar("New", evals1[1])
  axs[1].set_title("Accuracy")
  axs[1].bar("Old", evals2[1])
  axs[1].set_title("Accuracy")

  axs[2].bar("New", evals1[2])
  axs[2].set_title("Precision")
  axs[2].bar("Old", evals2[2])
  axs[2].set_title("Precision")

  axs[3].bar("New", evals1[3])
  axs[3].set_title("F1 Score")
  axs[3].bar("Old", evals2[3])
  axs[3].set_title("F1 Score")

  axs[4].bar("New", evals1[4])
  axs[4].set_title("Recall")
  axs[4].bar("Old", evals2[4])
  axs[4].set_title("Recall")

  axs[5].bar("New", evals1[5])
  axs[5].set_title("AUC")
  axs[5].bar("Old", evals2[5])
  axs[5].set_title("AUC")

  axs[6].bar("New", cros1)
  axs[6].set_title("Cross-Validation")
  axs[6].bar("Old", cros2)
  axs[6].set_title("Cross-Validation")

  return fig

And make tables for the models in terms of the metrics

In [ ]:
def make_figures_tables(path):
  final_training(arr_col, 10, "Colour").show()
  final_training(arr_cor, 10, "Asymmetry").show()
  
  best_random_forest(arr_col, 10, "Random Forest Colour").show()
  best_random_forest(arr_cor, 10, "Random Forest Asymmetry").show()

In [62]:
make_figures_tables(os.path.join(os.getcwd(), "output.csv"))